**Ingesta de Información de Correo de Clientes**

In [33]:
from pyspark.sql import functions as F

StatementMeta(sparkcapac, 154, 33, Finished, Available)

In [10]:
dfClienteCorreo = spark.read.option('header',True).csv('abfss://capacitacion@sesacapacitacion.dfs.core.windows.net/synapse/workspaces/synapsecapacitacion/warehouse/raw/GMolina/clientes_correos.csv')

StatementMeta(sparkcapac, 154, 10, Finished, Available)

In [11]:
dfClienteCorreo.show()

StatementMeta(sparkcapac, 154, 11, Finished, Available)

+-----------------+--------------------+
|     rowidcliente|               email|
+-----------------+--------------------+
|recTvJ1sICnY6e0ue|mario.dybvik@exam...|
|recV0UzVjes1sitxe|judy.paijmans@exa...|
|recWxdYj8g6rYlG00|alex.phillips@exa...|
|recaKpxSuMhgKdkpc| noah.li@example.com|
|recsx57ChLEoEZJh5|maritta.michler@e...|
|rec27Te7beTbnRT9a|ricardo.neal@exam...|
|recjAiAuBpUS1vaeU|reinhardt.sperlic...|
|recIa5U0Ht1kIGpve|antonio.vazquez@e...|
|rechqXqjqFmDwtXZ7|hilda.patterson@e...|
|recjEtLYlb6bwwRIU|imogen.hall@examp...|
|recPXwNxulSUAIlkr|sharon.guillot@ex...|
|recg28ai2lCvqVHwE|milla.hatala@exam...|
|reclKpXDCfQoP2dTn|peppi.maki@exampl...|
|reczpSdsaI5XV3wPH|maxim.lambert@exa...|
|recFXI4biywiNn4FU|josep.gonzalez@ex...|
|rec4N2pDW2bY1r38L|vitali.topp@examp...|
|recuUnU9zttIspmkO|nora.simpson@exam...|
|rec3u3hc3mwjRfZlq|william.bishop@ex...|
|recFzsWIlGec67GqW|victoria.deschamp...|
|recPI8Y8roYSHm7fw|bertha.dean@examp...|
+-----------------+--------------------+
only showing top

**Lectura de Datos Transaccionales**

Carga de archivos parquet con información de Cliente, Producto, Factura y FacturaProducto.

In [4]:
vPath = 'abfss://capacitacion@sesacapacitacion.dfs.core.windows.net/synapse/workspaces/synapsecapacitacion/warehouse/raw/GMolina/'
dfCliente = spark.read.load(vPath + 'cliente.parquet', format='parquet')
dfProducto = spark.read.load(vPath + 'producto.parquet', format='parquet')
dfFactura = spark.read.load(vPath + 'factura.parquet', format='parquet')
dfFactProducto = spark.read.load(vPath + 'facturaproducto.parquet', format='parquet')

StatementMeta(sparkcapac, 154, 4, Finished, Available)

**Cruce datos cliente con el correo**

In [14]:
dfClienteCorreo2 = ( dfCliente
    .join(dfClienteCorreo, dfCliente.rowidcliente == dfClienteCorreo.rowidcliente)
    .select(dfCliente.rowidcliente, dfClienteCorreo.email))

StatementMeta(sparkcapac, 154, 14, Finished, Available)

**Cruzar la información de todo el modelo**

In [19]:
dfResultado = ( dfFactura
    .join(dfFactProducto, dfFactura.rowidfactura == dfFactProducto.rowidfactura)
    .join(dfProducto,dfFactProducto.rowidproducto==dfProducto.rowidproducto )
    .select(dfFactura.rowidcliente, dfProducto.producto,dfFactura.fecha ))

StatementMeta(sparkcapac, 154, 19, Finished, Available)

**Conteo de Productos por cliente**

In [28]:
df_R1=dfResultado.groupby(dfResultado.rowidcliente,dfResultado.producto).count().withColumnRenamed("count", "ConteoP")

StatementMeta(sparkcapac, 154, 28, Finished, Available)

In [29]:
df_R1.show()

StatementMeta(sparkcapac, 154, 29, Finished, Available)

+-----------------+--------------------+-------+
|     rowidcliente|            producto|ConteoP|
+-----------------+--------------------+-------+
|recaJTnyKtGAI24fV|   Limpieza profunda|      2|
|recPI8Y8roYSHm7fw|    Limpieza express|      2|
|reccJcCj7FGHF2t9K|   Limpieza profunda|      2|
|recSERsMy4frAoYuL|   Limpieza profunda|      1|
|recJcRJ4O0oMtZCAv|    Limpieza express|      1|
|rec37Yb5EwFWulcwn|    Limpieza express|     14|
|recrK2xUGnyp2DSWb|    Muebles (puesto)|      3|
|rec0uQWYSnW1IvzcM|    Colchón 3 plazas|      1|
|recmwEnJXirC0pmUj|    Muebles (puesto)|      1|
|reca5K9yXb6KmDKtr|    Muebles (puesto)|      1|
|rec27Te7beTbnRT9a|   Limpieza profunda|      1|
|reccD6IC7H8UpOChp|   Limpieza profunda|      2|
|recAGExCpv7VM2fK4|   Limpieza profunda|      5|
|recdVKOcXa44Y8gZ2|    Limpieza express|      1|
|recDuTThryi7uX15s|     Colchón 1 plaza|      1|
|recHP6QPjizME0bgw|Colchón 2 1/2 plazas|      1|
|rec3Wp1wyHlLKt5IN|   Limpieza profunda|      1|
|reczFNN5c0p3big9v| 

**Sacar el máximo para obtener el producto más vendido**

In [37]:
df_R2= df_R1.groupBy("rowidcliente").agg(F.max("ConteoP")).withColumnRenamed("max(ConteoP)", "MaximoP")

StatementMeta(sparkcapac, 154, 37, Finished, Available)

**Resultado Final, donde se obtiene el listado de clientes con el producto más comprado.**

In [54]:
df_Resultado_Producto= (dfResultado.alias('t1')
    .join( df_R1.alias('t4'),(F.col('t1.rowidcliente') == F.col('t4.rowidcliente')))
    .join(df_R2.alias('t2'),(F.col('t1.rowidcliente') == F.col('t2.rowidcliente'))&(F.col('t4.ConteoP') == F.col('t2.MaximoP'))) 
    .join (dfClienteCorreo2.alias('t3'),(F.col('t2.rowidcliente') == F.col('t3.rowidcliente')))
    .select(F.col('t1.rowidcliente'),F.col('t1.producto'), F.col('t1.fecha'),F.col('t3.email'))
)

StatementMeta(sparkcapac, 154, 54, Finished, Available)

**Crear la tabla con el resultado final en el Pool**

In [56]:
df_Resultado_Producto.write.mode("overwrite").saveAsTable("default.tbl_gmolina")
vPathResult = 'abfss://capacitacion@sesacapacitacion.dfs.core.windows.net/synapse/workspaces/synapsecapacitacion/warehouse/raw/GMolina/tbl_gmolina.parquet'
df_Resultado_Producto.repartition(1).write.mode("overwrite").parquet(vPathResult)

StatementMeta(sparkcapac, 154, 56, Finished, Available)